
## Genetic Algorithm

The Genetic Algorithm (GA) is an evolutionary algorithm (EA) inspired by Charles Darwin’s theory of natural selection which espouses Survival of the fittest. As per the natural selection theory, the fittest individuals are selected to produce offsprings. The fittest parents' characteristics are then passed on to their offsprings using cross-over and mutation to ensure better chances of survival. Genetic algorithms are randomized search algorithms that generate high-quality optimization solutions by imitating the biologically inspired natural selection process such as selection, cross-over, and mutation.

##### Pseudocode:
- Initializing a random population.
- Running the population through a fitness function to return the best parents (highest accuracy).
- Selection from these best parents will occur depending on the n-parent parameter.
- These parents are then put through the crossover and mutation functions respectively.
- A new generation is created by selecting the fittest parents from the previous generation and applying cross-over and mutation.
- This process is repeated for a specified number of generations.

In [1]:
# spare cell to install packages

%pip install numpy


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import necessary libraries

import random
import numpy as np

In [3]:
# initialise parameters

population_size = 100
num_generations = 500
mutation_rate = 0.01
crossover_rate = 0.8


In [4]:
# space of the problem

grid = (10, 10)
num_objects = 10

In [5]:
# create a population of random permutations

def initialise_population():
    return [np.random.permutation(num_objects) for _ in range(population_size)]

In [6]:
# fitness function: minimise the sum of distances between consecutive objects in a layout -
# useful for scenarios where you want related or similar objects to be placed closer together, reducing the overall "travel" distance between them

def fitness_function(individual):
    # accumulate the total fitness score -
    # subtract distances to reduce the score as the sum of distances increases
    fitness = 0

    # iterate through consecutive pairs of objects in the individual list
    # obj1 and obj2 are two consecutive objects
    for i in range(len(individual) - 1):
        obj1, obj2 = individual[i], individual[i + 1]
        distance = abs(obj1 - obj2)
        
        # minimise distance
        fitness -= distance

    return fitness

In [7]:
# selection of best parents

def selection(population, fitnesses, n_parents):
    # pair each individual with its fitness score
    paired_population = list(zip(population, fitnesses))
    
    # paired population based on fitness scores in descending order
    paired_population.sort(key = lambda x: x[1], reverse=True)
    
    # select the top n_parents individuals
    selected_individuals = [individual for individual, fitness in paired_population[:n_parents]]
    
    return selected_individuals


In [8]:
# mutation functions

# single-point crossover
def crossover(parent1, parent2):

    # random crossover point is chosen between 1 and the length of parent
    point = random.randint(1, len(parent1) - 1)

    # children are created by swapping the 2 parts of the parents from crossover point
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    
    return child1, child2

# swap mutation
def mutate(individual):

    # 2 unique random indices are chosen from the range of the individual's length and swapped to introduce variation
    idx1, idx2 = random.sample(range(len(individual)), 2)
    individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

In [9]:
def genetic_algorithm():
    population = initialise_population()
    
    # main loop: algorithm runs for a fixed number of generations, evolving the population over time
    for generation in range(num_generations):

        # for each individual in the population, the fitness function is applied to evaluate how good that solution is --> result is a list of fitness scores
        fitnesses = [fitness_function(ind) for ind in population]
        
        # selection of best indiv (parents) --> used to create next generation
        parents = selection(population, fitnesses, n_parents=10)
        
        # create new generation
        new_population = []

        while len(new_population) < population_size:
            
            # 2 parents are randomly selected from the list of best parents
            parent1, parent2 = random.sample(parents, 2)

            # with a probability defined by crossover_rate, crossover is applied to the parents to create two children
            # if crossover does not occur, the children are simply copies of the parents
            if random.random() < crossover_rate:
                child1, child2 = crossover(parent1, parent2)
            else:
                child1, child2 = parent1, parent2

            # each child has a chance to be mutated based on the mutation_rate --> random changes to promote diversity
            if random.random() < mutation_rate:
                mutate(child1)
            if random.random() < mutation_rate:
                mutate(child2)

            # newly created children are added into population
            new_population.extend([child1, child2])
        
        population = new_population[:population_size]  # slicing to make population size constant, in case of extra children from loop

    # fitness of final population is evaluated, and bes solution is returned
    fitnesses = [fitness_function(ind) for ind in population]
    best_solution = population[np.argmax(fitnesses)]
    
    return best_solution


In [12]:
# run the algorithm

best_solution = genetic_algorithm()
print("Best solution:", best_solution)

Best solution: [1 1 1 2 2 3 5 6 8 9]


##### References
- https://huggingface.co/spaces/robitalhazmi/genetic_algorithm
- https://www.kaggle.com/code/zzettrkalpakbal/tutorial-of-genetic-algorithm/notebook#Parent-Selection
- ChatGPT